# consumer_tagger

Lee de kafka etiqueta y devuelve a kafka

1. Cargar los ficheros de diccionario de palabras
2. Para leer los mensajes nos suscribimos a el topic de Kafka, etiquetamos y enviamos a kafka nuevamente los mensajes etiquetados.

NOTA:  
Primero Ejecutar el consumer, asi, nos suscribirnos al topic de la cola de kafka y asi estamos esperando a leer los mensajes. Y ya luego el producer.   

In [1]:
# Imports
from pyspark.sql.types import *
from elasticsearch import Elasticsearch
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json


In [2]:
def load_simple_words():
    # cargamos el fichero de palabras y etiquetas desde parquet
    df = spark.read.load("simple_words.parquet")
    # Registrar el DataFrame como tabla.
    df.createOrReplaceTempView("simple_words")
    df.cache()
    print ("load simple_words OK")
    return df
        

df = spark.sql('SELECT *  FROM simple_words' )
df is None

In [3]:
topic_consumer='files_to_tag'
topic_producer='tagged_words'

consumer = KafkaConsumer(topic_consumer,bootstrap_servers=['localhost:9092'])
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])


#Cargamos diccionario con tags
load_simple_words()

for message in consumer:
    data = message.value
    file_param = message.key
    print (file_param)
    #print(data)
    countLinea = int(1)
    for linea in data.split("\n"):
            #print ("linea {0} : countLinea: {1} ".format(countLinea, linea))
            palabras = linea.split() # separar línea en palabras, por espacio en blanco
            start = int(0)
            for palabra in palabras:
                palabra = palabra.lower().strip(")(.,\n")
                #print palabra
                index_ini = linea.lower().index(palabra, start)
                index_end = int(index_ini+len(unicode(palabra, encoding='utf-8')))
                start = index_end
                #print (unicode(palabra, encoding='utf-8'), " datos: ", countLinea, index_ini ,index_end )
                simple_word_query =""
                simple_word_query = spark.sql('SELECT *  FROM simple_words WHERE lower(TOKEN) =="{}"'.format(palabra))
                tag =""
                if simple_word_query.count()>0: 
                    simple_word_query.show()
                    tag = simple_word_query.collect()[0].SEMTAG
                    #print (unicode(palabra, encoding='utf-8'), " datos: ", countLinea, index_ini ,index_end, etiqueta)                   
                    datos = {
                        'file':file_param,
                        'palabra': palabra,
                        'linea': countLinea,
                        'longitud': len(palabra),
                        'index_start':index_ini,
                        'index_end':index_end,
                        'tag':tag
                    }
                producer.send(topic_producer, key=file_param, value=datos) 
                json_str = json.dumps(datos)
                print('Datos en formato JSON:', json_str)
            countLinea = countLinea+1

load simple_words OK
quijote_test.txt
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   en|  prep| Z5 N3|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 2, "file": "quijote_test.txt", "index_start": 0, "tag": "Z5 N3", "linea": 1, "palabra": "en", "longitud": 2}')
+-----+-------+----------------+
|TOKEN| POSTAG|          SEMTAG|
+-----+-------+----------------+
|   un|    art|           Z5 N1|
|   un|num/art|Z5 N1 T3 T1.2 Z8|
+-----+-------+----------------+

('Datos en formato JSON:', '{"index_end": 5, "file": "quijote_test.txt", "index_start": 3, "tag": "Z5 N1", "linea": 1, "palabra": "un", "longitud": 2}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|lugar|  noun|    M7|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 11, "file": "quijote_test.txt", "index_start": 6, "tag": "M7", "linea": 1, "palabra": "lugar", "longitud": 5}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   de|  prep| 

+-----+------------+----------------+
|TOKEN|      POSTAG|          SEMTAG|
+-----+------------+----------------+
|  una|art/art/pron|Z5 N1 T3 T1.2 Z8|
|  una|num/art/pron|        Z5 Z8 N1|
+-----+------------+----------------+

('Datos en formato JSON:', '{"index_end": 19, "file": "quijote_test.txt", "index_start": 16, "tag": "Z5 N1 T3 T1.2 Z8", "linea": 3, "palabra": "una", "longitud": 3}')
('Datos en formato JSON:', '{"index_end": 19, "file": "quijote_test.txt", "index_start": 16, "tag": "Z5 N1 T3 T1.2 Z8", "linea": 3, "palabra": "una", "longitud": 3}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   de|  prep|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 27, "file": "quijote_test.txt", "index_start": 25, "tag": "Z5", "linea": 3, "palabra": "de", "longitud": 2}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
| algo|  pron| Z5 Z8|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 32, "file": "quijote_

+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   de|  prep|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 18, "file": "quijote_test.txt", "index_start": 16, "tag": "Z5", "linea": 6, "palabra": "de", "longitud": 2}')
('Datos en formato JSON:', '{"index_end": 18, "file": "quijote_test.txt", "index_start": 16, "tag": "Z5", "linea": 6, "palabra": "de", "longitud": 2}')
('Datos en formato JSON:', '{"index_end": 18, "file": "quijote_test.txt", "index_start": 16, "tag": "Z5", "linea": 6, "palabra": "de", "longitud": 2}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   de|  prep|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 37, "file": "quijote_test.txt", "index_start": 35, "tag": "Z5", "linea": 6, "palabra": "de", "longitud": 2}')
('Datos en formato JSON:', '{"index_end": 37, "file": "quijote_test.txt", "index_start": 35, "tag": "Z5", "linea": 6, "palabra": "de", "longitud": 2}')
+-----+------+----

+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   de|  prep|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 39, "file": "quijote_test.txt", "index_start": 37, "tag": "Z5", "linea": 8, "palabra": "de", "longitud": 2}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|  los|   art|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 43, "file": "quijote_test.txt", "index_start": 40, "tag": "Z5", "linea": 8, "palabra": "los", "longitud": 3}')
+--------+------+------+
|   TOKEN|POSTAG|SEMTAG|
+--------+------+------+
|cuarenta|   num| N1 T3|
+--------+------+------+

('Datos en formato JSON:', '{"index_end": 52, "file": "quijote_test.txt", "index_start": 44, "tag": "N1 T3", "linea": 8, "palabra": "cuarenta", "longitud": 8}')
+-----+------+--------+
|TOKEN|POSTAG|  SEMTAG|
+-----+------+--------+
|    y|  conj|Z5 A1.8+|
|    y|  noun|Z5 A1.8+|
+-----+------+--------+

('Datos en formato JSON:', '{"index_end":

('Datos en formato JSON:', '{"index_end": 51, "file": "quijote_test.txt", "index_start": 44, "tag": "S2.1 Z5 A9", "linea": 10, "palabra": "nuestro", "longitud": 7}')
+-----+------+-------+
|TOKEN|POSTAG| SEMTAG|
+-----+------+-------+
|  Con| pnoun|     Z2|
|  con|  prep|Z5 A4.1|
+-----+------+-------+

('Datos en formato JSON:', '{"index_end": 63, "file": "quijote_test.txt", "index_start": 60, "tag": "Z2", "linea": 10, "palabra": "con", "longitud": 3}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|  los|   art|    Z5|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 67, "file": "quijote_test.txt", "index_start": 64, "tag": "Z5", "linea": 10, "palabra": "los", "longitud": 3}')
+---------+-------+------+
|    TOKEN| POSTAG|SEMTAG|
+---------+-------+------+
|cincuenta|adj/num|    N1|
+---------+-------+------+

('Datos en formato JSON:', '{"index_end": 77, "file": "quijote_test.txt", "index_start": 68, "tag": "N1", "linea": 10, "palabra": "cincuenta

+------+------+---------------+
| TOKEN|POSTAG|         SEMTAG|
+------+------+---------------+
|alguna|   adj|         Z5 S2f|
|alguna|  pron|N5 S2.1-f Z8 Z5|
+------+------+---------------+

('Datos en formato JSON:', '{"index_end": 23, "file": "quijote_test.txt", "index_start": 17, "tag": "Z5 S2f", "linea": 13, "palabra": "alguna", "longitud": 6}')
+----------+------+--------+
|     TOKEN|POSTAG|  SEMTAG|
+----------+------+--------+
|diferencia|  noun|A6.1- N2|
+----------+------+--------+

('Datos en formato JSON:', '{"index_end": 34, "file": "quijote_test.txt", "index_start": 24, "tag": "A6.1- N2", "linea": 13, "palabra": "diferencia", "longitud": 10}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+
|   en|  prep| Z5 N3|
+-----+------+------+

('Datos en formato JSON:', '{"index_end": 37, "file": "quijote_test.txt", "index_start": 35, "tag": "Z5 N3", "linea": 13, "palabra": "en", "longitud": 2}')
+-----+------+------+
|TOKEN|POSTAG|SEMTAG|
+-----+------+------+

+-----+------+------------+
|TOKEN|POSTAG|      SEMTAG|
+-----+------+------------+
|   no|   adv|S8- Z4 Z6 Z5|
+-----+------+------------+

('Datos en formato JSON:', '{"index_end": 70, "file": "quijote_test.txt", "index_start": 68, "tag": "S8- Z4 Z6 Z5", "linea": 15, "palabra": "no", "longitud": 2}')
+-----+------+----------+
|TOKEN|POSTAG|    SEMTAG|
+-----+------+----------+
|   se|  pron|Z5 Z8 S1.1|
+-----+------+----------+

('Datos en formato JSON:', '{"index_end": 73, "file": "quijote_test.txt", "index_start": 71, "tag": "Z5 Z8 S1.1", "linea": 15, "palabra": "se", "longitud": 2}')
('Datos en formato JSON:', '{"index_end": 73, "file": "quijote_test.txt", "index_start": 71, "tag": "Z5 Z8 S1.1", "linea": 15, "palabra": "se", "longitud": 2}')
+-----+-------+----------------+
|TOKEN| POSTAG|          SEMTAG|
+-----+-------+----------------+
|   un|    art|           Z5 N1|
|   un|num/art|Z5 N1 T3 T1.2 Z8|
+-----+-------+----------------+

('Datos en formato JSON:', '{"index_end": 82

KeyboardInterrupt: 